In [1]:
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [2]:
DATA_DIR = '/home/HDD6TB/datasets/emotions/ABAW/ABAW_8/8th_ABAW_Annotations'

In [3]:
import os
from PIL import Image
import cv2
import pathlib

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn import svm,metrics,preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score,roc_auc_score,average_precision_score
import mord

from collections import defaultdict
import os
import glob
import gc
import random
import numpy as np
from tqdm import tqdm
import time
import pickle
import pandas as pd
import random
import csv  
import matplotlib.pyplot as plt

In [4]:
def get_names(id):
    name = ""
    if id>=0 and id<10:
        name = "0000" + str(id)
    elif id>=10 and id<100:
        name = "000" + str(id)
    elif id>=100 and id<1000:
        name = "00" + str(id)
    elif id>=1000 and id<10000:
        name = "0" + str(id)
    else:
        name = str(id)
    return name

In [5]:
compare_filenames=lambda x: int(os.path.splitext(x)[0])

## PyTorch

In [6]:
import glob
from itertools import chain
import os
import random
import zipfile

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from tqdm.notebook import tqdm

print(f"Torch: {torch.__version__}")
device = 'cuda'
use_cuda = torch.cuda.is_available()
print(use_cuda)

Torch: 2.0.1+cu118
True


In [7]:
#num_classes=7
if False:
    #PATH='affectnet_vggface2_enet2_gmp_smooth.pt'
    PATH='enet_b2_8_best.pt'
    #PATH='enet_b2_7.pt'
    IMG_SIZE=260 #224 #
else:
    #PATH='affectnet_vggface2_enet0.pt'
    #PATH='affectnet_vggface2_enet0_new.pt'
    #PATH='enet_b0_7.pt'
    #PATH='enet_b0_8_best_afew.pt'
    PATH='enet_b0_8_best_vgaf.pt'
    #PATH='mobilevit_mtl_new.pt'
    
    #PATH='enet_b0_8_va_mtl.pt'
    IMG_SIZE=224
    
#IMG_SIZE=112

test_transforms = transforms.Compose(
    [
        transforms.Resize((IMG_SIZE,IMG_SIZE)),
        #transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    ]
)
np_transforms = transforms.Compose(
    [
        transforms.ToPILImage(None),
        transforms.Resize((IMG_SIZE,IMG_SIZE)),
        #transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    ]
)


In [6]:
print(PATH)
feature_extractor_model = torch.load('../../../models/affectnet_emotions/'+PATH)

enet_b0_8_best_vgaf.pt


In [8]:
if True:
    classifier_weights=feature_extractor_model.classifier[0].weight.cpu().data.numpy()
    classifier_bias=feature_extractor_model.classifier[0].bias.cpu().data.numpy()
elif False:
    classifier_weights=feature_extractor_model.classifier.weight.cpu().data.numpy()
    classifier_bias=feature_extractor_model.classifier.bias.cpu().data.numpy()
elif False:
    classifier_weights=feature_extractor_model.head.fc.weight.cpu().data.numpy()
    classifier_bias=feature_extractor_model.head.fc.bias.cpu().data.numpy()
else:
    classifier_weights=feature_extractor_model.fc.weight.cpu().data.numpy()
    classifier_bias=feature_extractor_model.fc.bias.cpu().data.numpy()
print(classifier_weights.shape,classifier_weights)
print(classifier_bias.shape,classifier_bias)

(10, 512) [[ 0.01794354 -0.03431091 -0.00016818 ... -0.0268224  -0.02347743
   0.02176051]
 [-0.02611435 -0.02214147  0.03573207 ... -0.00466314  0.04239551
  -0.01990408]
 [-0.05963856 -0.01730311  0.04917104 ... -0.01539477 -0.00746309
  -0.01494454]
 ...
 [ 0.04475736 -0.00453137  0.0154746  ... -0.00426385 -0.00999709
  -0.01408496]
 [ 0.0183069  -0.00611857 -0.00720661 ...  0.00141637 -0.01378473
  -0.00045164]
 [ 0.01398357  0.00257701  0.00391316 ... -0.00689993 -0.0030514
   0.03322613]]
(10,) [ 0.00227628  0.00355737 -0.00111303 -0.03025234 -0.0463375   0.0778885
  0.01492943 -0.01847218 -0.05123357  0.05881068]


In [9]:
if True:
    feature_extractor_model.classifier=torch.nn.Identity()
elif False:
    feature_extractor_model.head.fc=torch.nn.Identity()
else:
    feature_extractor_model.fc=torch.nn.Identity()
feature_extractor_model=feature_extractor_model.to(device)
feature_extractor_model.eval()

DDAMNet(
  (features): Sequential(
    (0): Conv_block(
      (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (prelu): PReLU(num_parameters=64)
    )
    (1): Conv_block(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (prelu): PReLU(num_parameters=64)
    )
    (2): Mix_Depth_Wise(
      (conv): Conv_block(
        (conv): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (prelu): PReLU(num_parameters=128)
      )
      (conv_dw): MDConv(
        (mixed_depthwise_conv): ModuleList(
          (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=64, bias=False)
    

In [10]:
def get_probab(features):
    x=np.dot(features,np.transpose(classifier_weights))+classifier_bias
    return x

## Cropped

In [11]:
print(test_transforms)
img_names=[]
X_global_features=[]
imgs=[]
data_dir=os.path.join(DATA_DIR,'cropped')
print(data_dir)
for filename in tqdm(os.listdir(data_dir)):
    frames_dir=os.path.join(data_dir,filename)   
    if not os.path.isdir(frames_dir):
        continue
    for img_name in os.listdir(frames_dir):
        if img_name.lower().endswith('.jpg'):
            img = Image.open(os.path.join(frames_dir,img_name))
            img_tensor = test_transforms(img)
            if img.size:
                img_names.append(filename+'/'+img_name)
                imgs.append(img_tensor)
                if len(imgs)>=64: #96: #64: #32:        
                    features,_,_  = feature_extractor_model(torch.stack(imgs, dim=0).to(device))
                    features=features.data.cpu().numpy()
                    #print(features.shape)

                    if len(X_global_features)==0:
                        X_global_features=features
                    else:
                        X_global_features=np.concatenate((X_global_features,features),axis=0)
                    imgs=[]

if len(imgs)>0:        
    features,_,_  = feature_extractor_model(torch.stack(imgs, dim=0).to(device))
    features=features.data.cpu().numpy()

    if len(X_global_features)==0:
        X_global_features=features
    else:
        X_global_features=np.concatenate((X_global_features,features),axis=0)

    imgs=[]

    #X_scores=X_global_features #get_probab(X_global_features)
    #print(X_global_features.shape,X_scores.shape)

Compose(
    Resize(size=(112, 112), interpolation=bilinear, max_size=None, antialias=warn)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)
/home/HDD6TB/datasets/emotions/ABAW/ABAW_6/6th_ABAW_Annotations/cropped


  0%|          | 0/594 [00:00<?, ?it/s]

----------------------------------

In [12]:
X_scores=get_probab(X_global_features)

In [14]:
filename2featuresAll={img_name:(global_features,scores) for img_name,global_features,scores in zip(img_names,X_global_features,X_scores)}
print(len(filename2featuresAll))
#cropped 2942996, aligned 2941546

2942996


### Save/load features 

In [6]:
import pickle
num_classes=8
#model_name='enet_b2_8_best'
model_name='enet_b0_8_best_vgaf'
#model_name='mobilevit_mtl'
#model_name='mbf_va'

#model_name='ddamfnet_8' #affectnet8_epoch4_acc0.6462
#model_name='ddamfnet_8_mtl' #affectnet8_epoch6_acc0

#model_name='enet_b0_8_va_mtl'
#MODEL2FEATURES=model_name+'_aligned_112.pickle'
MODEL2FEATURES=model_name+'_cropped.pickle' 

print(MODEL2FEATURES)

enet_b0_8_best_vgaf_cropped.pickle


In [16]:
if False:
    with open(MODEL2FEATURES, 'wb') as handle:
        pickle.dump(filename2featuresAll, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [7]:
#filename=MODEL2FEATURES
filename='../abaw5/'+MODEL2FEATURES
with open(filename, 'rb') as handle:
    filename2featuresAll=pickle.load(handle)
print(filename,len(filename2featuresAll))

../abaw5/enet_b0_8_best_vgaf_cropped.pickle 2942996


In [8]:
if True:
    for filename in filename2featuresAll:
        filename2featuresAll[filename]=(filename2featuresAll[filename][0].astype(np.float16),filename2featuresAll[filename][1].astype(np.float16))

# Audios

In [8]:
d=os.path.join(DATA_DIR,'../../videos')
video2len={}
for filename in os.listdir(d):
    fn, ext = os.path.splitext(os.path.basename(filename))
    vid=os.path.join(d,filename)
    cap = cv2.VideoCapture(vid)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    video2len[fn]=total_frames+1 #FIX ME!!! NEED TO ADD 1 to the number of frames for consistency with challeng's organizer
    print(fn,video2len[fn])

27-60-1280x720 2933
video39 10679
77-30-1280x720 5673
91-30-1920x1080 10922
360 3510
136 4488
197 1870
291 1831
16-30-1920x1080 5474
video2 2065
89-30-1080x1920 631
408 2979
110-30-270x480 9979
324 5615
14-30-1920x1080 5311
129-24-1280x720 6154
153 3678
314 2261
255 1692
284 4055
298 2837
206 4477
video84 526
387 2912
286 2678
55-25-1280x720 1506
336 4345
video52 10680
60-30-1920x1080 4565
120 3592
267 3842
121-24-1920x1080 10629
349 4055
46-30-484x360 2945
218 3850
video55 8420
368 2930
9-15-1920x1080 47435
384 3452
228 2156
video3 7802
106-30-720x1280 1405
64-24-640x360 4919
244 2322
28-30-1280x720-2 397
235 3683
video86_3 403
92-24-1920x1080 8164
376 3570
185 4053
157 2605
38-30-1920x1080 2432
video87 1823
57-25-426x240 301
4-30-1920x1080 8261
video86_2 887
video45_5 415
143 10103
114 2450
243 4121
133 461
video64 6101
125-25-1280x720 6842
video81 8434
274 3527
113 15021
video66 155
171 3977
446 2249
126-30-1080x1920 4595
168 2979
video33 7554
201 1361
video61 117
236 3674
119 4577


In [9]:
d=os.path.join(DATA_DIR,'../../new_vids')
#video2len={}
for filename in os.listdir(d):
    fn, ext = os.path.splitext(os.path.basename(filename))
    vid=os.path.join(d,filename)
    cap = cv2.VideoCapture(vid)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    video2len[fn]=total_frames+1 #FIX ME!!! NEED TO ADD 1 to the number of frames for consistency with challeng's organizer
    print(fn,video2len[fn])

454 27789
471 5934
479 6080
463 22511
461 7677
453 4946
495 2907
478 10190
499 6564
459 3686
475 9559
483 14632
491 10212
489 11980
493 871
465 25064
455 779
480 4809
477 8236
497 6328
468 10229
500 3120
469 18249
474 7953
494 1700
496 7233
451 9125
458 23633
481 4701
482 6331
486 20200
498 2200
452 11545
457 14957
467 15186
464 14578
462 3758
473 10352
488 6017
476 8260
470 9102
487 11919
485 9481
456 6303
490 7312
472 9750
460 9766
466 22919
492 510
484 3596


In [6]:
import wave, struct
from fnmatch import fnmatch
video_dir=os.path.join(DATA_DIR,'../../new_vids')
audio_dir=os.path.join(DATA_DIR,'../../new_vids_audios')
for filename in os.listdir(video_dir):
    print(filename)
    video_path=os.path.join(video_dir,filename)
    fn=os.path.splitext(filename)[0]
    waveFile = os.path.join(audio_dir, fn+'.wav')
    command = "ffmpeg -i "+video_path+" -ac 1 -ar 16000 -vn "+waveFile
    #print(command)
    os.system(command=command)

454.mp4
471.avi
479.avi
463.avi
461.mp4
453.avi
495.avi
478.avi
499.avi
459.avi
475.avi
483.mp4
491.avi
489.avi
493.avi
465.avi
455.avi
480.avi
477.avi
497.mp4
468.mp4
500.avi
469.mp4
474.avi
494.mp4
496.avi
451.avi
458.avi
481.avi
482.avi
486.avi
498.avi
452.avi
457.avi
467.mp4
464.avi
462.avi
473.avi
488.avi
476.avi
470.avi
487.avi
485.avi
456.avi
490.avi
472.avi
460.mp4
466.avi
492.mp4
484.avi


---------

In [1]:
from transformers import Wav2Vec2Processor, Wav2Vec2Model,AutoProcessor, HubertModel
import torchaudio
import torch
if False:
    processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
    model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")
else:
    processor = AutoProcessor.from_pretrained("facebook/hubert-large-ls960-ft")
    model = HubertModel.from_pretrained("facebook/hubert-large-ls960-ft")
model.eval()

/usr/local/lib/python3.8/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


HubertModel(
  (feature_extractor): HubertFeatureEncoder(
    (conv_layers): ModuleList(
      (0): HubertLayerNormConvLayer(
        (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
        (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (activation): GELUActivation()
      )
      (1-4): 4 x HubertLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
        (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (activation): GELUActivation()
      )
      (5-6): 2 x HubertLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,))
        (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (activation): GELUActivation()
      )
    )
  )
  (feature_projection): HubertFeatureProjection(
    (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (projection): Linear(in_features=512, out_features=1024, bias=True)
    (dropout): Dropout(p=

In [7]:
AUDIO_DIR=DATA_DIR+'/../../audios'
videoname2audiofeatures={}
for filename in sorted(os.listdir(AUDIO_DIR)):
    videoname,_=os.path.splitext(filename)
    array, fs = torchaudio.load(os.path.join(AUDIO_DIR,filename))
    inp = processor(array.squeeze(), sampling_rate=fs, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inp)
    videoname2audiofeatures[videoname]=outputs.last_hidden_state.numpy()[0]
    print(videoname,fs,videoname2audiofeatures[videoname].shape,inp['input_values'].shape[1]/len(videoname2audiofeatures[videoname]))
    break

1-30-1280x720 16000 (17259, 768) 320.0136160843618


In [ ]:
import librosa

AUDIO_DIR=DATA_DIR+'/../../audios'
videoname2audiofeatures={}
for filename in tqdm(sorted(os.listdir(AUDIO_DIR))):
    videoname,_=os.path.splitext(filename)
    filepath=os.path.join(AUDIO_DIR,filename)
    fs=librosa.get_samplerate(filepath)
    stream = librosa.stream(
        filepath,
        block_length=360,
        frame_length=16000,
        hop_length=16000
    )
    total_outputs=None
    for speech in stream:
        if len(speech.shape) > 1:
            speech = speech[:, 0] + speech[:, 1]

        inp = processor(speech, sampling_rate=16000, return_tensors="pt")
        with torch.no_grad():
            outputs = model(**inp)
        outputs=outputs.last_hidden_state.numpy()[0]
        if total_outputs is None:
            total_outputs=outputs
        else:
            total_outputs=np.concatenate((total_outputs,outputs),axis=0)
    frames_count=video2len[videoname]
    audio_scale=len(total_outputs)/frames_count
    audio_features=np.zeros((frames_count,total_outputs.shape[1]))
    for frame_number in range(frames_count):
        ind=int(frame_number*audio_scale)
        audio_features[frame_number]=total_outputs[ind]
    
    videoname2audiofeatures[videoname]=audio_features.astype(np.float16)
    print(videoname,fs,videoname2audiofeatures[videoname].shape,total_outputs.shape)

  0%|          | 1/548 [02:22<21:35:18, 142.08s/it]

1-30-1280x720 16000 (10356, 1024) (17259, 1024)


  0%|          | 2/548 [02:33<9:52:33, 65.12s/it]  

10-60-1280x720 16000 (2502, 1024) (4167, 1024)


  1%|          | 3/548 [02:45<6:11:35, 40.91s/it]

...

109-30-1280x720 16000 (1046, 1024) (1747, 1024)


  3%|▎         | 19/548 [17:27<4:58:58, 33.91s/it]

11-24-1920x1080 16000 (5390, 1024) (8982, 1024)


  4%|▎         | 20/548 [19:42<9:23:47, 64.07s/it]

110-30-270x480 16000 (9979, 1024) (16632, 1024)


In [7]:
#MODEL2AUDIOFEATURES='../abaw6/affwild2_wav2vec2.pickle'
MODEL2AUDIOFEATURES='affwild2_hubert.pickle'

if False:
    with open(MODEL2AUDIOFEATURES, 'wb') as handle:
        pickle.dump(videoname2audiofeatures, handle, protocol=pickle.HIGHEST_PROTOCOL)
else:
    with open(MODEL2AUDIOFEATURES, 'rb') as handle:
        videoname2audiofeatures=pickle.load(handle)
print(len(videoname2audiofeatures),videoname2audiofeatures['video7'].shape)

KeyError: 'video7'

In [ ]:
import librosa

AUDIO_DIR=os.path.join(DATA_DIR,'../../new_vids_audios')
#videoname2audiofeatures={}
for filename in tqdm(sorted(os.listdir(AUDIO_DIR))):
    videoname,_=os.path.splitext(filename)
    filepath=os.path.join(AUDIO_DIR,filename)
    fs=librosa.get_samplerate(filepath)
    stream = librosa.stream(
        filepath,
        block_length=300, #360
        frame_length=16000,
        hop_length=16000
    )
    total_outputs=None
    for speech in stream:
        if len(speech.shape) > 1:
            speech = speech[:, 0] + speech[:, 1]

        inp = processor(speech, sampling_rate=16000, return_tensors="pt")
        with torch.no_grad():
            outputs = model(**inp)
        outputs=outputs.last_hidden_state.numpy()[0]
        if total_outputs is None:
            total_outputs=outputs
        else:
            total_outputs=np.concatenate((total_outputs,outputs),axis=0)
    frames_count=video2len[videoname]
    audio_scale=len(total_outputs)/frames_count
    audio_features=np.zeros((frames_count,total_outputs.shape[1]))
    for frame_number in range(frames_count):
        ind=int(frame_number*audio_scale)
        audio_features[frame_number]=total_outputs[ind]
    
    videoname2audiofeatures[videoname]=audio_features
    print(videoname,fs,videoname2audiofeatures[videoname].shape,total_outputs.shape)

In [ ]:
#filename2audio_features={}
for videoname in videoname2audiofeatures:
    audio_features=videoname2audiofeatures[videoname]
    for i,af in enumerate(audio_features):
        imagename=videoname+'/'+get_names(i+1)+'.jpg'
        filename2audio_features[imagename]=af
print(len(filename2audio_features))

In [9]:
MODEL2AUDIOFEATURES='../abaw6/affwild2_file2wav2vec2.pickle'
#MODEL2AUDIOFEATURES='affwild2_file2hubert.pickle'

if False:
    with open(MODEL2AUDIOFEATURES, 'wb') as handle:
        pickle.dump(filename2audio_features, handle, protocol=pickle.HIGHEST_PROTOCOL)
else:
    with open(MODEL2AUDIOFEATURES, 'rb') as handle:
        filename2audio_features=pickle.load(handle)
print(len(filename2audio_features))

3091661


In [10]:
if True:
    for filename in filename2audio_features:
        filename2audio_features[filename]=filename2audio_features[filename].astype(np.float16)

# Classifiers

In [17]:
import tensorflow as tf
from tensorflow.keras.models import Model,Sequential, load_model,model_from_json
from tensorflow.keras.applications import mobilenet,mobilenet_v2,densenet,inception_resnet_v2,inception_v3,vgg16,resnet_v2,resnet
#from tensorflow.keras.applications import efficientnet as enet
import efficientnet.tfkeras as enet
#from tensorflow.keras.utils.generic_utils import CustomObjectScope
from tensorflow.keras.layers import Flatten, Dense, Dropout,GlobalAveragePooling2D,Activation, Conv2D, Reshape,DepthwiseConv2D,Input
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, Callback, CSVLogger, EarlyStopping
from tensorflow.keras.metrics import top_k_categorical_accuracy
import numpy as np
from sklearn.metrics import confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import tensorflow.keras.backend as K 

print(tf.__version__)
from tensorflow.compat.v1.keras.backend import set_session 
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess=tf.compat.v1.Session(config=config)
set_session(sess)

2.13.1


In [18]:
from copy import deepcopy
class SaveBestModel(tf.keras.callbacks.Callback):
    def __init__(self, save_best_metric='val_loss', this_max=False):
        self.save_best_metric = save_best_metric
        self.max = this_max
        if this_max:
            self.best = float('-inf')
        else:
            self.best = float('inf')

    def on_epoch_end(self, epoch, logs=None):
        metric_value = logs[self.save_best_metric]
        if self.max:
            if metric_value > self.best:
                self.best = metric_value
                self.best_model_weights = deepcopy(self.model.get_weights())

        else:
            if metric_value < self.best:
                self.best = metric_value
                self.best_model_weights = deepcopy(self.model.get_weights())

In [19]:
def one_hot_transfer(label,class_num):
    return np.eye(class_num)[label]

def metric_for_Exp(gt,pred,class_num=8):
    # compute_acc
    acc = accuracy_score(gt,pred)
    # compute_F1
    gt = one_hot_transfer(gt,class_num)
    pred = one_hot_transfer(pred,class_num)
    F1 = []
    for i in range(class_num):
        gt_ = gt[:,i]
        pred_ = pred[:,i]
        F1.append(f1_score(gt_.flatten(), pred_))
    F1_mean = np.mean(F1)
    return F1_mean,acc,F1

In [20]:
def CCC_score(x, y):
    vx = x - np.mean(x)
    vy = y - np.mean(y)
    rho = np.sum(vx * vy) / (np.sqrt(np.sum(vx**2)) * np.sqrt(np.sum(vy**2)))
    x_m = np.mean(x)
    y_m = np.mean(y)
    x_s = np.std(x)
    y_s = np.std(y)
    ccc = 2*rho*x_s*y_s/(x_s**2 + y_s**2 + (x_m - y_m)**2)
    return ccc

def metric_for_VA(gt_V,gt_A,pred_V,pred_A):
    ccc_V,ccc_A = CCC_score(gt_V,pred_V),CCC_score(gt_A,pred_A)
    return ccc_V,ccc_A, 0.5*(ccc_V+ccc_A)

def CCC_numpy(y_true, y_pred):
    '''Reference numpy implementation of Lin's Concordance correlation coefficient'''
    
    # covariance between y_true and y_pred
    s_xy = np.cov([y_true, y_pred])[0,1]
    # means
    x_m = np.mean(y_true)
    y_m = np.mean(y_pred)
    # variances
    s_x_sq = np.var(y_true)
    s_y_sq = np.var(y_pred)
    
    # condordance correlation coefficient
    ccc = (2.0*s_xy) / (s_x_sq + s_y_sq + (x_m-y_m)**2)
    
    return ccc

def CCC(y_true, y_pred):
    '''Lin's Concordance correlation coefficient: https://en.wikipedia.org/wiki/Concordance_correlation_coefficient
    
    The concordance correlation coefficient is the correlation between two variables that fall on the 45 degree line through the origin.
    
    It is a product of
    - precision (Pearson correlation coefficient) and
    - accuracy (closeness to 45 degree line)

    Interpretation:
    - `rho_c =  1` : perfect agreement
    - `rho_c =  0` : no agreement
    - `rho_c = -1` : perfect disagreement 
    
    Args: 
    - y_true: ground truth
    - y_pred: predicted values
    
    Returns:
    - concordance correlation coefficient (float)
    '''
    
    
    # covariance between y_true and y_pred
    #N = K.int_shape(y_pred)[-1]
    #s_xy = 1.0 / (N - 1.0 + K.epsilon()) * K.sum((y_true - K.mean(y_true)) * (y_pred - K.mean(y_pred)))
    #s_xy = K.mean(K.sum((y_true - K.mean(y_true)) * (y_pred - K.mean(y_pred))))
    s_xy = K.mean((y_true - K.mean(y_true)) * (y_pred - K.mean(y_pred)))
    # means
    x_m = K.mean(y_true)
    y_m = K.mean(y_pred)
    # variances
    s_x_sq = K.var(y_true)
    s_y_sq = K.var(y_pred)
    
    # condordance correlation coefficient
    ccc = (2.0*s_xy) / (s_x_sq + s_y_sq + (x_m-y_m)**2+K.epsilon())
    #print(s_xy,s_x_sq,s_y_sq,x_m,y_m)
    return ccc

def CCC_VA(y_true, y_pred):
    return 1-0.5*(CCC(y_true[:,0], y_pred[:,0])+CCC(y_true[:,1], y_pred[:,1]))


In [21]:
import math
def f1_score_max_for_AU_one_class(gt, pred, thresh,type=0):
    gt = gt[:,type]
    pred = pred[:,type]
    P = []
    R = []
    ACC = []
    F1 = []
    for i in thresh:
        new_pred = ((pred >= i) * 1).flatten()
        P.append(precision_score(gt.flatten(), new_pred))
        R.append(recall_score(gt.flatten(), new_pred))
        ACC.append(accuracy_score(gt.flatten(), new_pred))
        F1.append(f1_score(gt.flatten(), new_pred))

    F1_MAX = max(F1)
    if F1_MAX < 0 or math.isnan(F1_MAX):
        F1_MAX = 0
        F1_THRESH = 0
        accuracy = 0
    else:
        idx_thresh = np.argmax(F1)
        F1_THRESH = thresh[idx_thresh]
        accuracy = ACC[idx_thresh]
    return F1,F1_MAX,F1_THRESH,accuracy

def f1_score_max(gt, pred, thresh,c=12):
    F1_s = []
    F1_t = []
    ACC = []
    from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
    for i in range(c):
        F1, F1_MAX, F1_THRESH,acc = f1_score_max_for_AU_one_class(gt,pred,thresh,i)
        F1_s.append(F1_MAX)
        F1_t.append(F1_THRESH)
        ACC.append(acc)
    F1_s=np.array(F1_s)
    F1_t=np.array(F1_t)
    ACC=np.array(ACC)
    return F1_s.mean(),F1_t.mean(),F1_s,F1_t,ACC

## Expr 

In [16]:
idx_to_class={0: 'Angry', 1: 'Contempt', 2: 'Disgust', 3: 'Fear', 4: 'Happy', 5: 'Neutral', 6: 'Sad', 7: 'Surprise'}
class_to_idx={cls:idx for idx,cls in idx_to_class.items()}
print('AffectNet:',class_to_idx)

#idx_to_class_2={0: 'Neutral', 1:'Angry', 2:'Disgust', 3:'Fear', 4:'Happy', 5:'Sad', 6:'Surprised', 7:'Other'} #ABAW
#idx_to_class={0: 'Angry', 1: 'Contempt', 2: 'Disgust', 3: 'Fear', 4: 'Happy', 5: 'Neutral', 6: 'Sad', 7: 'Surprised'} #AffectNet
AFFECTNET2MTL=np.array([5,0,2,3,4,6,7,1])
print(AFFECTNET2MTL)

AffectNet: {'Angry': 0, 'Contempt': 1, 'Disgust': 2, 'Fear': 3, 'Happy': 4, 'Neutral': 5, 'Sad': 6, 'Surprise': 7}
[5 0 2 3 4 6 7 1]


In [94]:
def get_image2ExprAudio(dirname):
    dirpath=os.path.join(DATA_DIR,'EXPR_Recognition_Challenge/',dirname)
    num_missed=0
    X,y=[],[]
    for filename in os.listdir(dirpath):
        fn, ext = os.path.splitext(os.path.basename(filename))
        fn_short=fn
        if fn.endswith('_left'):
            fn_short=fn[:-5]
        elif fn.endswith('_right'):
            fn_short=fn[:-6]
        if ext.lower()=='.txt':
            with open(os.path.join(dirpath,filename)) as f:
                lines = f.read().splitlines()
                for i,line in enumerate(lines):
                    if i>0:
                        expression=int(line)
                        if expression>=0:
                            imagename_short=fn_short+'/'+get_names(i)+'.jpg'
                            imagename=fn+'/'+get_names(i)+'.jpg'
                            has_image=imagename_short in filename2audio_features
                            if dirname=='Validation_Set':
                                has_frame=os.path.exists(os.path.join(DATA_DIR,'cropped',imagename))
                                if has_image:
                                    has_image=has_frame
                                elif has_frame:
                                    #print(imagename,imagename in filename2audio_features,imagename in filename2featuresAll)
                                    imagename_short=fn_short+'/'+get_names(i-1)+'.jpg'
                                    has_image=imagename_short in filename2audio_features
                            if has_image:
                                X.append(filename2audio_features[imagename_short])
                                y.append(expression)
                            else:
                                num_missed+=1
                                #print(imagename)
    X=np.array(X).astype(np.float16)
    y=np.array(y)
    print(X.shape,y.shape,num_missed)
    return X,y

X_train,y_train=get_image2ExprAudio('Train_Set')
X_val,y_val=get_image2ExprAudio('Validation_Set')
TRAIN_VAL=False
batch_size=256 #128

(597357, 768) (597357,) 4
(280532, 768) (280532,) 3698


In [85]:
def get_image2Expr(dirname):
    dirpath=os.path.join(DATA_DIR,'EXPR_Recognition_Challenge/',dirname)
    num_missed=0
    X,y=[],[]
    for filename in os.listdir(dirpath):
        fn, ext = os.path.splitext(os.path.basename(filename))
        if ext.lower()=='.txt':
            with open(os.path.join(dirpath,filename)) as f:
                lines = f.read().splitlines()
                for i,line in enumerate(lines):
                    if i>0:
                        expression=int(line)
                        if expression>=0:
                            imagename=fn+'/'+get_names(i)+'.jpg'
                            if imagename in filename2featuresAll:
                                X.append(filename2featuresAll[imagename][0])
                                #X.append(filename2featuresAll[imagename][1])
                                y.append(expression)
                            else:
                                num_missed+=1
    X=np.array(X)
    y=np.array(y)
    print(X.shape,y.shape,num_missed)
    return X,y

X_train,y_train=get_image2Expr('Train_Set')
X_val,y_val=get_image2Expr('Validation_Set')
TRAIN_VAL=False
batch_size=256 #128

(585317, 1280) (585317,) 12044
(280532, 1280) (280532,) 3698


In [95]:
if False:
    TRAIN_VAL=True
    X_train=np.concatenate((X_train,X_val))
    y_train=np.concatenate((y_train,y_val))
    print(X_train.shape,y_train.shape)

(877889, 768) (877889,)


In [42]:
def print_expr(X=X_val,y=y_val):
    y_val_preds=mlpModel.predict(X,verbose=0)
    y_pred=np.argmax(y_val_preds,axis=1)
    print('Acc:',(y_pred==y).mean(), 'F1:',f1_score(y_true=y,y_pred=y_pred, average="macro"))
    print(metric_for_Exp(y,y_pred))

In [87]:
(unique, counts) = np.unique(y_train, return_counts=True)
num_classes=len(unique)
cw=1/counts
cw/=cw.min()
class_weights = {i:cwi for i,cwi in zip(unique,cw)}
print(counts, class_weights, num_classes, unique)
print(np.unique(y_val, return_counts=True))


[259456  22699  16067  17488 129974 103908  43947 272310] {0: 1.0495421188949188, 1: 11.99656372527424, 2: 16.948403560092117, 3: 15.571248856358647, 4: 2.095111329958299, 5: 2.6206836817184436, 6: 6.19632739436139, 7: 1.0} 8 [0 1 2 3 4 5 6 7]
(array([0, 1, 2, 3, 4, 5, 6, 7]), array([ 82258,   6126,   5296,   8408,  34511,  25157,  12332, 106444]))


In [96]:
mlpModel=Sequential()
num_classes=8
if False:
    mlpModel.add(Dense(num_classes, input_shape=X_val.shape[1:],activation='softmax',use_bias=True,kernel_regularizer=tf.keras.regularizers.l2(1.0/batch_size)))
else:
    mlpModel.add(Dense(128, input_shape=X_val.shape[1:],activation='relu')) #256
    #mlpModel.add(Dense(128, input_shape=(1280,),activation='relu')) #256
    mlpModel.add(Dense(num_classes,activation='softmax'))

In [97]:
gc.collect()

45467

In [98]:
mlpModel.compile(optimizer=Adam(learning_rate=1e-3), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
mlpModel.summary()

save_best_model = SaveBestModel('val_accuracy',True)
mlpModel.fit(X_train,y_train, batch_size=batch_size, epochs=2 if TRAIN_VAL else 10, verbose=1, shuffle=True,
             callbacks=[save_best_model], validation_data=(X_val,y_val),class_weight=class_weights)
best_model_weights = save_best_model.best_model_weights
print(save_best_model.best)

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 128)               98432     
                                                                 
 dense_11 (Dense)            (None, 8)                 1032      
                                                                 
Total params: 99464 (388.53 KB)
Trainable params: 99464 (388.53 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/2
3430/3430 [==============================] - 22s 6ms/step - loss: 3.9801 - accuracy: 0.3498 - val_loss: 1.4831 - val_accuracy: 0.4485
Epoch 2/2
3430/3430 [==============================] - 21s 6ms/step - loss: 3.3271 - accuracy: 0.4734 - val_loss: 1.3881 - val_accuracy: 0.5027
0.5026805996894836


In [99]:
if True:
    #mlpModel.save_weights('expr_enet0_vgaf_train_val_new.h5')
    mlpModel.save_weights('expr_wav2vec_train_val_new.h5')
elif True:
    mlpModel.load_weights('expr_enet0_vgaf_train_val.h5')

In [43]:
if True:
    print_expr()
    print('Best weights:')
    mlpModel.set_weights(best_model_weights)
print_expr()

Acc: 0.4474498452939415 F1: 0.3534379977160974
(0.3534379977160974, 0.4474498452939415, [0.5050543210026993, 0.13419216317767044, 0.560565275908479, 0.036674410886159976, 0.4447339182406805, 0.4692188778757843, 0.2012156400254014, 0.4758493746119046])
Best weights:
Acc: 0.4859766443756862 F1: 0.3771706981857585
(0.3771706981857585, 0.4859766443756862, [0.571038931925175, 0.17630948305374874, 0.47358237833010863, 0.03297045101088646, 0.4677675659099146, 0.5217275333970527, 0.27579590676520754, 0.49817333509397427])


In [23]:
if True:
    #mlpModel.save_weights('expr_enet0_vgaf.h5')#Acc: 0.48984429583790795 F1: 0.3828262567771098
    mlpModel.save_weights('expr_wav2vec2.h5')#Acc: 0.410110076568805 F1: 0.29090239611199764
else:
    #mlpModel.load_weights('../expr_enet0_vgaf.h5') #Acc: 0.500285172458044 F1: 0.3807067519117502
    mlpModel.load_weights('../abaw5/expr_enet0_vgaf.h5') #Acc: 0.49544436998274705 F1: 0.3844313672414089
    print_expr()

In [46]:
mlpModel.load_weights('../abaw5/expr_enet0_vgaf.h5')
y_val_preds=mlpModel.predict(X_val,verbose=0)
y_pred=np.argmax(y_val_preds,axis=1)
print('Acc:',(y_pred==y_val).mean(), 'F1:',f1_score(y_true=y_val,y_pred=y_pred, average="macro"))
print(metric_for_Exp(y_val,y_pred))

Acc: 0.49544793463847264 F1: 0.38445558136937796
(0.38445558136937796, 0.49544793463847264, [0.5975372028718637, 0.18703154959037824, 0.5204209286923246, 0.017722567287784676, 0.47926124957534205, 0.48604448320976884, 0.28961248144521995, 0.4980141882823414])


enet_b0_8_best_vgaf
aligned
112
Acc: 0.36997808749986594 F1: 0.21352704404593095
(0.21352704404593095, 0.36997808749986594, [0.45700583162587527, 0.07635619242579324, 0.003076194983435873, 0.009468317552804079, 0.34928095010272137, 0.21187660892709106, 0.1647308567096285, 0.4364214000400981])

../expr_enet0_vgaf.h5
Acc: 0.20843684874655494 F1: 0.0910746772554131


224
Acc: 0.47395701146384794 F1: 0.3045351324629094
(0.3045351324629094, 0.47395701146384794, [0.5518323928131282, 0.11104639875752281, 0.028970239662891757, 0.029782898346265387, 0.42825757737346165, 0.5113845517526626, 0.2515561703388993, 0.5234508306584433])

../expr_enet0_vgaf.h5
Acc: 0.42862351608048643 F1: 0.3395506137533413


cropped
Acc: 0.49544436998274705 F1: 0.3844313672414089
(0.3844313672414089, 0.49544436998274705, [0.5975792617828053, 0.18688981868898186, 0.5203826342899192, 0.01771963235902956, 0.4792554615273745, 0.4860550818814191, 0.289585043895661, 0.4979840035060809])
../expr_enet0_vgaf.h5
Acc: 0.500285172458044 F1: 0.3807067519117502
(0.3807067519117502, 0.500285172458044, [0.608643679809698, 0.15128569174948728, 0.5162256615077384, 0.01599360255897641, 0.4778312968664514, 0.4611138100075078, 0.3029224246739377, 0.5116378481202044])


enet_b0_8_va_mtl
aligned
112
Acc: 0.36310406828978836 F1: 0.21416588541497295
(0.21416588541497295, 0.36310406828978836, [0.5104795189391993, 0.1258546760922471, 0.010939722131057872, 0.009548019096038193, 0.33523095946425735, 0.18541315145321896, 0.19094959313671342, 0.3449114430070513])

../expr_enet0_vgaf.h5
Acc: 0.17444566379147022 F1: 0.09599142979803564


224

cropped
Acc: 0.44668344431294826 F1: 0.3355545672821115
(0.3355545672821115, 0.44668344431294826, [0.510145234564414, 0.16641097373883224, 0.3162162162162162, 0.0352055352055352, 0.5013185136754931, 0.4799950349621416, 0.2053190850693402, 0.4698259448249195])
../expr_enet0_vgaf.h5
Acc: 0.4230711647869049 F1: 0.30920724751192785


mobilevit
cropped
224
Acc: 0.46147320091825533 F1: 0.3560574896064487
(0.3560574896064487, 0.46147320091825533, [0.5428998117005389, 0.15102462545204065, 0.44947762979503947, 0.12488323632966876, 0.47030709895410744, 0.40287070773986966, 0.20392039203920392, 0.5030764148411208])

mbf_va
cropped
112
Acc: 0.46184392511371253 F1: 0.32738624577811665
(0.32738624577811665, 0.46184392511371253, [0.5275200664106117, 0.13163163163163163, 0.4120108948382383, 0.02459419576979833, 0.47229383600629454, 0.3058296783625731, 0.21955208437635595, 0.5256575788294297])

aligned
Acc: 0.41698808574829577 F1: 0.2605325777277917
(0.2605325777277917, 0.41698808574829577, [0.5104076268095435, 0.08551319648093843, 0.03246321784527765, 0.04267053701015965, 0.4294620130290349, 0.2965347390252333, 0.2095684295882512, 0.47764086203389466])


enet_b2_best
Acc: 0.4377147705074644 F1: 0.32046977297192947
(0.32046977297192947, 0.4377147705074644, [0.5279750265857089, 0.18667230205691746, 0.32060923369823896, 0.022576492722051686, 0.457802371541502, 0.3702839197046216, 0.23979880986115049, 0.438040027605245])


ddamfnet_8
Acc: 0.43266721800008556 F1: 0.30837374615523644
(0.30837374615523644, 0.43266721800008556, [0.5215368741365138, 0.1129685025514693, 0.3387109850020268, 0.05675889328063242, 0.4707923787344315, 0.29412395223701415, 0.21788596803993387, 0.4542124152598697])

ddamfnet_8_mtl
Acc: 0.4619865113427345 F1: 0.33872014801701744
(0.33872014801701744, 0.4619865113427345, [0.5691907822979275, 0.1422720413882302, 0.2694043321299639, 0.06790817145609246, 0.5095350810593541, 0.4453414505029116, 0.25683655334518146, 0.449272771956478])

Acc: 0.46895184863045924 F1: 0.351079601079747
(0.351079601079747, 0.46895184863045924, [0.5672248538336185, 0.0777629408530359, 0.43501871284949, 0.08656489442592268, 0.4938491272202964, 0.41942100601370563, 0.256280299691494, 0.4725149737504132])


wav2vec2
Acc: 0.410110076568805 F1: 0.29090239611199764
(0.29090239611199764, 0.410110076568805, [0.4758486812936623, 0.047472429043273805, 0.5612936880542514, 0.15109275511960077, 0.13438885152032565, 0.3119477387658142, 0.10830172777075431, 0.5368732973282989])

hubert:


In [22]:
mlpModel.load_weights('../abaw6/expr_wav2vec2.h5')
y_val_expr_preds_audio=mlpModel.predict(X_val,verbose=0)
#y_val_expr_preds_audio=y_val_preds

y_pred=np.argmax(y_val_expr_preds_audio,axis=1)
print('Acc:',(y_pred==y_val).mean(), 'F1:',f1_score(y_true=y_val,y_pred=y_pred, average="macro"))

Acc: 0.4101172058802561 F1: 0.2908597184585734


In [20]:
if False:
    with open('y_val_expr_preds_audio.pickle', 'wb') as handle:
        pickle.dump(y_val_expr_preds_audio, handle, protocol=pickle.HIGHEST_PROTOCOL)
else:
    with open('y_val_expr_preds_audio.pickle', 'rb') as handle:
        y_val_expr_preds_audio=pickle.load(handle)
print(y_val_expr_preds_audio.shape)
y_pred=np.argmax(y_val_expr_preds_audio,axis=1)
print('Acc:',(y_pred==y_val).mean(), 'F1:',f1_score(y_true=y_val,y_pred=y_pred, average="macro"))

(280532, 8)
Acc: 0.410110076568805 F1: 0.29090239611199764


In [21]:
for w in np.linspace(0,1,11):
    print(w)
    y_ensemble=w*y_val_preds+(1-w)*y_val_expr_preds_audio
    y_pred=np.argmax(y_ensemble,axis=1)
    print('Acc:',(y_pred==y_val).mean(), 'F1:',f1_score(y_true=y_val,y_pred=y_pred, average="macro"))
    

0.0
Acc: 0.410110076568805 F1: 0.29090239611199764
0.1
Acc: 0.43592887798896385 F1: 0.3141889443420316
0.2
Acc: 0.46409678753226014 F1: 0.3439200559542131
0.30000000000000004
Acc: 0.4903932528196427 F1: 0.3723581978456718
0.4
Acc: 0.5081238503985285 F1: 0.3910699591929091
0.5
Acc: 0.5203149729799096 F1: 0.4030154928467773
0.6000000000000001
Acc: 0.5178197139720245 F1: 0.4033380362792486
0.7000000000000001
Acc: 0.5115958250752143 F1: 0.3989305928956265
0.8
Acc: 0.5050725050974577 F1: 0.39334207090252027
0.9
Acc: 0.49983959049235027 F1: 0.3883340065997216
1.0
Acc: 0.49544436998274705 F1: 0.3844313672414089


### Scores 

In [20]:
dirpath=os.path.join(DATA_DIR,'EXPR_Recognition_Challenge/Validation_Set')
num_missed=0
y_scores_all,y_val=[],[]

for filename in os.listdir(dirpath):
    fn, ext = os.path.splitext(os.path.basename(filename))
    if ext.lower()=='.txt':
        with open(os.path.join(dirpath,filename)) as f:
            lines = f.read().splitlines()
            prev_val=None
            for i,line in enumerate(lines):
                if i>0:
                    expression=int(line)
                    #if expression>=0:
                    if expression>=0:
                        imagename=fn+'/'+get_names(i)+'.jpg'
                        if imagename in filename2featuresAll:
                            scores=filename2featuresAll[imagename][1]
                            if False:
                                scores=scores[:-2]
                            y_scores_all.append(scores[AFFECTNET2MTL])
                            y_val.append(expression)
                        else:
                            num_missed+=1
y_scores_all=np.array(y_scores_all)
y_val=np.array(y_val)
print(y_scores_all.shape,y_val.shape,num_missed)

y_scores=np.argmax(y_scores_all[y_val!=7,:7],axis=1)
print('w/o Other',metric_for_Exp(y_val[y_val!=7],y_scores,7))
y_scores=np.argmax(y_scores_all,axis=1)
print('with Other:',metric_for_Exp(y_val,y_scores,8))

(280532, 8) (280532,) 3698
w/o Other (0.38315520007157966, 0.5219142043104636, [0.5941131445271728, 0.11673050704408232, 0.2563962686160057, 0.1146315414738341, 0.6221359518916038, 0.574566439059762, 0.40351254788859714])
with Other: (0.2574340560684253, 0.32562417121754383, [0.47261988187005705, 0.08159349444926674, 0.16895322458679918, 0.08013937282229966, 0.462785267165087, 0.4738317407716553, 0.28230688149375094, 0.037242585388486384])


embeddings:
Acc: 0.4961929476851126 F1: 0.3822393283107972
(0.3822393283107972, 0.4961929476851126, [0.5816358825689002, 0.19165239340124404, 0.4775824331003319, 0.06514293702383449, 0.4966704252920027, 0.4404618154856478, 0.27396940324652574, 0.5307993363678905])

In [21]:
#y_val_preds=mlpModel.predict(X_val,verbose=0)
y_pred=np.argmax(y_val_preds,axis=1)
print('Acc:',(y_pred==y_val).mean(), 'F1:',f1_score(y_true=y_val,y_pred=y_pred, average="macro"))
print(metric_for_Exp(y_val,y_pred))

Acc: 0.49544436998274705 F1: 0.3844313672414089
(0.3844313672414089, 0.49544436998274705, [0.5975792617828053, 0.18688981868898186, 0.5203826342899192, 0.01771963235902956, 0.4792554615273745, 0.4860550818814191, 0.289585043895661, 0.4979840035060809])


In [25]:
def scores2probabs(x):
    e_x = np.exp(x - np.max(x))#[np.newaxis])
    e_x = e_x / e_x.sum(axis=1)[:,None]
    return e_x

In [22]:
y_probabs_all=scores2probabs(y_scores_all)
print(y_probabs_all)

[[1.24244377e-01 3.35698426e-02 3.62046719e-01 ... 3.47988039e-01
  1.85106192e-02 2.83648036e-02]
 [1.08953334e-01 3.82212065e-02 3.71574074e-01 ... 3.60739708e-01
  1.69553831e-02 2.36208811e-02]
 [1.04377650e-01 3.58219035e-02 3.63524914e-01 ... 3.76680374e-01
  1.79313309e-02 2.13590246e-02]
 ...
 [9.49333087e-02 1.18450960e-02 1.37146823e-02 ... 8.56873155e-01
  1.38443382e-03 2.01118737e-03]
 [5.32542281e-02 9.44525655e-03 1.81043278e-02 ... 9.00745928e-01
  5.90948097e-04 8.60838452e-04]
 [1.20561913e-01 1.66836120e-02 1.92864668e-02 ... 8.15231144e-01
  1.07049837e-03 1.43829477e-03]]


In [23]:
#y_scores=np.argmax(y_scores_all[y_val!=7,:7],axis=1)
y_pred_scores=np.argmax(y_probabs_all[:,:7],axis=1)

for t in np.linspace(0,1,11):
    y_pred_all=np.argmax(y_val_preds,axis=1)
    criterion=np.array([y_p[y_s]>t for y_p,y_s in zip(y_probabs_all,y_pred_scores)])
    #criterion=y_pred_all!=7
    y_pred_all[criterion]=y_pred_scores[criterion]
    print(t,metric_for_Exp(y_val,y_pred_all,8),criterion.sum())

0.0 (0.25318654349366876, 0.32388105456774985, [0.4769210717411138, 0.07942046910199213, 0.16737295680353265, 0.08012607830126078, 0.4677859391395593, 0.47200883604909466, 0.2818569968127969, 0.0]) 280532
0.1 (0.25326521499390486, 0.32394521837080975, [0.4769440824179234, 0.07981262153084674, 0.16737295680353265, 0.08012607830126078, 0.4679755022608895, 0.47201469383702754, 0.2818569968127969, 1.878798696113705e-05]) 280462
0.2 (0.2537128102109227, 0.32443001154948453, [0.47691840013896725, 0.08047557783594339, 0.1676061362825544, 0.08006305484111839, 0.46909122173501844, 0.4717336839233002, 0.2819764819103185, 0.0018379250201609125]) 279373
0.30000000000000004 (0.2646285128636536, 0.33743387563629107, [0.4819260656917997, 0.08364286696932177, 0.17715617715617715, 0.08296426222319667, 0.4743061909118633, 0.4758355916892502, 0.28597466508703184, 0.05522228318058766]) 266226
0.4 (0.29731775443903014, 0.37889438638016343, [0.5108383904282426, 0.09784817381336945, 0.21327354260089687, 0.08

---------------

### Smooth validation predictions  

In [22]:
if True:
    mlpModel=Sequential()
    mlpModel.add(Dense(128, input_shape=(1280,),activation='relu')) #256
    mlpModel.add(Dense(8,activation='softmax'))
    mlpModel.load_weights('../ABAW5/expr_enet0_vgaf.h5')


In [23]:
data_dir=os.path.join(DATA_DIR,'faces')
dirpath=os.path.join(DATA_DIR,'EXPR_Recognition_Challenge/Validation_Set')
test_videos={}
for filename in tqdm(os.listdir(dirpath)):
    fn, ext = os.path.splitext(os.path.basename(filename))
    if ext.lower()=='.txt':
        X,indices,expressions,scores=[],[],[],[]
        with open(os.path.join(dirpath,filename)) as f:
            lines = f.read().splitlines()
            prev_val=None
            for i,line in enumerate(lines):
                if i>0:
                    imagename=fn+'/'+get_names(i)+'.jpg'
                    if imagename in filename2featuresAll:
                        X.append(filename2featuresAll[imagename][0])
                        indices.append(i)
                        expressions.append(int(line))
                        scores.append(filename2featuresAll[imagename][1][AFFECTNET2MTL])
        test_videos[fn]=(mlpModel.predict(np.array(X),verbose=0),indices,np.array(expressions),np.array(scores))
print(len(test_videos))

100%|██████████| 70/70 [00:41<00:00,  1.70it/s]

70


In [22]:
hyperparams=[(isMean,delta) for delta in [0,2,7]  for isMean in [1,0] if not (isMean==0 and delta==0)]
total_true=[]
total_preds=[[] for _ in range(len(hyperparams))]
for videoname,(y_pred_expr,indices,expressions,scores) in test_videos.items():
    for i,ind in enumerate(indices):
        if expressions[i]>=0:
            total_true.append(expressions[i])
    cur_ind=0
    preds_proba=[]
    for i in range(indices[-1]):
        if indices[cur_ind]-1==i:
            preds_proba.append(y_pred_expr[cur_ind])
            cur_ind+=1
        else:
            if cur_ind==0:
                preds_proba.append(y_pred_expr[cur_ind])
            else:
                w=(i-indices[cur_ind-1]+1)/(indices[cur_ind]-indices[cur_ind-1])
                pred=w*y_pred_expr[cur_ind-1]+(1-w)*y_pred_expr[cur_ind]
                preds_proba.append(pred)
    
    preds_proba=np.array(preds_proba)
    for hInd,(isMean,delta) in enumerate(hyperparams):
        preds=[]
        for i in range(len(preds_proba)):
            i1=max(i-delta,0)
            if isMean:
                proba=np.mean(preds_proba[i1:i+delta+1],axis=0)
            else:
                proba=np.median(preds_proba[i1:i+delta+1],axis=0)
            preds.append(np.argmax(proba))
        for i,ind in enumerate(indices):
            if expressions[i]>=0:
                total_preds[hInd].append(preds[ind-1])

total_true=np.array(total_true)
for hInd,(isMean,delta) in enumerate(hyperparams):
    preds=np.array(total_preds[hInd])
    print('mean' if isMean else 'median',delta,'Acc:',(preds==total_true).mean(), 'F1:',f1_score(y_true=total_true,y_pred=preds, average="macro"))

mean 0 Acc: 0.4377147705074644 F1: 0.32046977297192947
mean 2 Acc: 0.4502053241697917 F1: 0.3308365959569308
median 2 Acc: 0.4485655825360387 F1: 0.3292047135773658
mean 7 Acc: 0.46036815764333483 F1: 0.33783569428081117
median 7 Acc: 0.4585787004691087 F1: 0.3357647927368673


In [21]:
#deltas=[0,1,5,7,15]
deltas=[0,5,15,25,50,100,200,500]
total_true=[]
total_preds=[[] for _ in range(len(deltas))]
for videoname,(y_pred_expr,indices,expressions,scores) in test_videos.items():
    for i,ind in enumerate(indices):
        if expressions[i]>=0:
            total_true.append(expressions[i])
    cur_ind=0
    preds_proba=[]
    for i in range(indices[-1]):
        if indices[cur_ind]-1==i:
            preds_proba.append(y_pred_expr[cur_ind])
            cur_ind+=1
        else:
            if cur_ind==0:
                preds_proba.append(y_pred_expr[cur_ind])
            else:
                w=1-(i-indices[cur_ind-1]+1)/(indices[cur_ind]-indices[cur_ind-1])
                pred=w*y_pred_expr[cur_ind-1]+(1-w)*y_pred_expr[cur_ind]
                preds_proba.append(pred)
    
    preds_proba=np.array(preds_proba)
    for hInd,delta in enumerate(deltas):
        preds=[]
        for i in range(len(preds_proba)):
            i1=max(i-delta,0)
            proba=np.mean(preds_proba[i1:i+delta+1],axis=0)
            preds.append(np.argmax(proba))
        for i,ind in enumerate(indices):
            if expressions[i]>=0:
                total_preds[hInd].append(preds[ind-1])

total_true=np.array(total_true)
for hInd,delta in enumerate(deltas):
    preds=np.array(total_preds[hInd])
    print(delta,'Acc:',(preds==total_true).mean(), 'F1:',f1_score(y_true=total_true,y_pred=preds, average="macro"))

0 Acc: 0.49544436998274705 F1: 0.3844313672414089
5 Acc: 0.5147327221136982 F1: 0.4004462320756203
15 Acc: 0.5290448148517816 F1: 0.41233892932451965
25 Acc: 0.5351189882081189 F1: 0.41821548069619524
50 Acc: 0.5424194031340454 F1: 0.4236679637523476
100 Acc: 0.5408010494346456 F1: 0.41968206198832886
200 Acc: 0.539617583733763 F1: 0.4195667528908445
500 Acc: 0.530777237534399 F1: 0.4032242437980253


0 Acc: 0.49544436998274705 F1: 0.3844313672414089
5 Acc: 0.5147255928022472 F1: 0.40041647458913604
15 Acc: 0.5290234269174283 F1: 0.41233375265784433
25 Acc: 0.5352187985684342 F1: 0.41830667436931435
50 Acc: 0.542572683330244 F1: 0.4236698679610459
100 Acc: 0.5408224373689989 F1: 0.4197549134058517
200 Acc: 0.5397387820284317 F1: 0.41970605922798776
500 Acc: 0.5307736728786734 F1: 0.4031952312654022

In [25]:
deltas=[0,5,15,25,50,100,200,500]
total_true=[]
total_preds=[[] for _ in range(len(deltas))]
for videoname,(y_pred_expr,indices,expressions,scores) in test_videos.items():
    for i,ind in enumerate(indices):
        if expressions[i]>=0:
            total_true.append(expressions[i])

    pretrained_scores_probabs=scores2probabs(scores)
    cur_ind=0
    preds_proba,pretrained_preds_proba=[],[]
    for i in range(indices[-1]):
        if indices[cur_ind]-1==i:
            preds_proba.append(y_pred_expr[cur_ind])
            pretrained_preds_proba.append(pretrained_scores_probabs[cur_ind])
            cur_ind+=1
        else:
            if cur_ind==0:
                preds_proba.append(y_pred_expr[cur_ind])
                pretrained_preds_proba.append(pretrained_scores_probabs[cur_ind])
            else:
                w=1-(i-indices[cur_ind-1]+1)/(indices[cur_ind]-indices[cur_ind-1])
                pred=w*y_pred_expr[cur_ind-1]+(1-w)*y_pred_expr[cur_ind]
                preds_proba.append(pred)
                
                s=w*pretrained_scores_probabs[cur_ind-1]+(1-w)*pretrained_scores_probabs[cur_ind]
                pretrained_preds_proba.append(s)
    
    preds_proba=np.array(preds_proba)
    pretrained_preds_proba=np.array(pretrained_preds_proba)
    
    for hInd,delta in enumerate(deltas):
        preds=[]
        for i in range(len(preds_proba)):
            i1=max(i-delta,0)
            proba=np.mean(preds_proba[i1:i+delta+1],axis=0)
            pred=np.argmax(proba)
            
            pretrained_proba=np.mean(pretrained_preds_proba[i1:i+delta+1],axis=0)
            pretrained_pred=np.argmax(pretrained_proba[:7])
            if pretrained_proba[pretrained_pred]>0.8:
                pred=pretrained_pred
            
            preds.append(pred)
        for i,ind in enumerate(indices):
            if expressions[i]>=0:
                total_preds[hInd].append(preds[ind-1])

total_true=np.array(total_true)
for hInd,delta in enumerate(deltas):
    preds=np.array(total_preds[hInd])
    print(delta,'Acc:',(preds==total_true).mean(), 'F1:',f1_score(y_true=total_true,y_pred=preds, average="macro"))

0 Acc: 0.49767940912266695 F1: 0.39420142699710226
5 Acc: 0.5209886929120385 F1: 0.41445194418026476
15 Acc: 0.5361776909586072 F1: 0.4275109020140136
25 Acc: 0.5422768169050233 F1: 0.4330901402834371
50 Acc: 0.5462549726947371 F1: 0.4343184315522313
100 Acc: 0.5418383642507807 F1: 0.4260578994232742
200 Acc: 0.5382202386893474 F1: 0.41917871154835207
500 Acc: 0.5281180043631386 F1: 0.40132874798491425


In [52]:
thresholds=np.linspace(0,1,11)

delta=50
total_true=[]
total_preds=[[] for _ in range(len(thresholds))]
for videoname,(y_pred_expr,indices,expressions,scores) in test_videos.items():
    for i,ind in enumerate(indices):
        if expressions[i]>=0:
            total_true.append(expressions[i])

    pretrained_scores_probabs=scores2probabs(scores)
    cur_ind=0
    preds_proba,pretrained_preds_proba=[],[]
    for i in range(indices[-1]):
        if indices[cur_ind]-1==i:
            preds_proba.append(y_pred_expr[cur_ind])
            pretrained_preds_proba.append(pretrained_scores_probabs[cur_ind])
            cur_ind+=1
        else:
            if cur_ind==0:
                preds_proba.append(y_pred_expr[cur_ind])
                pretrained_preds_proba.append(pretrained_scores_probabs[cur_ind])
            else:
                w=1-(i-indices[cur_ind-1]+1)/(indices[cur_ind]-indices[cur_ind-1])
                pred=w*y_pred_expr[cur_ind-1]+(1-w)*y_pred_expr[cur_ind]
                preds_proba.append(pred)
                
                s=w*pretrained_scores_probabs[cur_ind-1]+(1-w)*pretrained_scores_probabs[cur_ind]
                pretrained_preds_proba.append(s)
    
    preds_proba=np.array(preds_proba)
    pretrained_preds_proba=np.array(pretrained_preds_proba)
    
    for hInd,t in enumerate(thresholds):
        preds=[]
        for i in range(len(preds_proba)):
            i1=max(i-delta,0)
            proba=np.mean(preds_proba[i1:i+delta+1],axis=0)
            pred=np.argmax(proba)
            
            pretrained_proba=np.mean(pretrained_preds_proba[i1:i+delta+1],axis=0)
            pretrained_pred=np.argmax(pretrained_proba[:7])
            if pretrained_proba[pretrained_pred]>t:
                pred=pretrained_pred
            
            preds.append(pred)
        for i,ind in enumerate(indices):
            if expressions[i]>=0:
                total_preds[hInd].append(preds[ind-1])

total_true=np.array(total_true)
for hInd,t in enumerate(thresholds):
    preds=np.array(total_preds[hInd])
    print(t,'Acc:',(preds==total_true).mean(), 'F1:',f1_score(y_true=total_true,y_pred=preds, average="macro"))

0.0 Acc: 0.3547901843639941 F1: 0.2768972468723374
0.1 Acc: 0.3547901843639941 F1: 0.2768972468723374
0.2 Acc: 0.3553961758373376 F1: 0.2774307894069533
0.30000000000000004 Acc: 0.3920693539417963 F1: 0.3091697700675562
0.4 Acc: 0.4600009981036032 F1: 0.3626075313169596
0.5 Acc: 0.5122374631058132 F1: 0.4070930183009853
0.6000000000000001 Acc: 0.5351902813226299 F1: 0.43217040242047583
0.7000000000000001 Acc: 0.5428828083783668 F1: 0.43835013947562684
0.8 Acc: 0.546247843383286 F1: 0.43428887590742027
0.9 Acc: 0.5458165200404945 F1: 0.4282005950922094
1.0 Acc: 0.5424158384783198 F1: 0.42366109129065965


### Audio 

In [27]:
if False:
    mlpModelAudio=mlpModel
else:
    mlpModelAudio=Sequential()
    mlpModelAudio.add(Dense(128, input_shape=(768,),activation='relu')) #256
    mlpModelAudio.add(Dense(8,activation='softmax'))
    mlpModelAudio.load_weights('../abaw6/expr_wav2vec2.h5')


In [28]:
data_dir=os.path.join(DATA_DIR,'faces')
dirpath=os.path.join(DATA_DIR,'EXPR_Recognition_Challenge/Validation_Set')
test_audios={}
for filename in tqdm(os.listdir(dirpath)):
    fn, ext = os.path.splitext(os.path.basename(filename))
    fn_short=fn
    if fn.endswith('_left'):
        fn_short=fn[:-5]
    elif fn.endswith('_right'):
        fn_short=fn[:-6]
    if ext.lower()=='.txt':
        X,indices,expressions,scores=[],[],[],[]
        with open(os.path.join(dirpath,filename)) as f:
            lines = f.read().splitlines()
            prev_val=None
            for i,line in enumerate(lines):
                if i>0:
                    imagename=fn+'/'+get_names(i)+'.jpg'
                    imagename_short=fn_short+'/'+get_names(i)+'.jpg'
                    imagename=fn+'/'+get_names(i)+'.jpg'
                    has_image=imagename_short in filename2audio_features
                    has_frame=os.path.exists(os.path.join(DATA_DIR,'cropped',imagename))
                    if has_image:
                        has_image=has_frame
                    elif has_frame:
                        #print(imagename,imagename in filename2audio_features,imagename in filename2featuresAll)
                        imagename_short=fn_short+'/'+get_names(i-1)+'.jpg'
                        has_image=imagename_short in filename2audio_features
                    if has_image:
                        X.append(filename2audio_features[imagename_short])
                        indices.append(i)
                        expressions.append(int(line))
                        scores.append(filename2featuresAll[imagename][1][AFFECTNET2MTL])
        test_audios[fn]=(mlpModelAudio.predict(np.array(X),verbose=0),indices,np.array(expressions),np.array(scores))
print(len(test_audios))


100%|██████████| 70/70 [00:39<00:00,  1.78it/s]

70


In [55]:
#deltas=[0,1,5,7,15]
deltas=[0,5,15,25,50,100,200,500]
total_true=[]
total_preds=[[] for _ in range(len(deltas))]
for videoname,(y_pred_expr,indices,expressions,scores) in test_audios.items():
    for i,ind in enumerate(indices):
        if expressions[i]>=0:
            total_true.append(expressions[i])
    cur_ind=0
    preds_proba=[]
    for i in range(indices[-1]):
        if indices[cur_ind]-1==i:
            preds_proba.append(y_pred_expr[cur_ind])
            cur_ind+=1
        else:
            if cur_ind==0:
                preds_proba.append(y_pred_expr[cur_ind])
            else:
                w=1-(i-indices[cur_ind-1]+1)/(indices[cur_ind]-indices[cur_ind-1])
                pred=w*y_pred_expr[cur_ind-1]+(1-w)*y_pred_expr[cur_ind]
                preds_proba.append(pred)
    
    preds_proba=np.array(preds_proba)
    for hInd,delta in enumerate(deltas):
        preds=[]
        for i in range(len(preds_proba)):
            i1=max(i-delta,0)
            proba=np.mean(preds_proba[i1:i+delta+1],axis=0)
            preds.append(np.argmax(proba))
        for i,ind in enumerate(indices):
            if expressions[i]>=0:
                total_preds[hInd].append(preds[ind-1])

total_true=np.array(total_true)
for hInd,delta in enumerate(deltas):
    preds=np.array(total_preds[hInd])
    print(delta,'Acc:',(preds==total_true).mean(), 'F1:',f1_score(y_true=total_true,y_pred=preds, average="macro"))

0 Acc: 0.410110076568805 F1: 0.29090239611199764
5 Acc: 0.4806474840659889 F1: 0.33263223831368627
15 Acc: 0.5046732636561961 F1: 0.34164236744431886
25 Acc: 0.513214178774614 F1: 0.3457739134483474
50 Acc: 0.5193703392126389 F1: 0.3521084905477545
100 Acc: 0.5205038997333638 F1: 0.35453723394331516
200 Acc: 0.5206037100936791 F1: 0.3551542666808812
500 Acc: 0.5236229734932201 F1: 0.3595042852144297


In [40]:
#Ensemble
deltas=[0,5,15,25,50,100,200,500]
for weight in np.linspace(0,1,11):
    total_true=[]
    total_preds=[[] for _ in range(len(deltas))]
    for videoname,(y_pred_expr,indices,expressions,_) in test_videos.items():
        y_pred_expr_audio,_,_,_=test_audios[videoname]
        for i,ind in enumerate(indices):
            if expressions[i]>=0:
                total_true.append(expressions[i])
        cur_ind=0
        preds_proba=[]
        y_ensemble=weight*y_pred_expr + (1-weight)*y_pred_expr_audio
        for i in range(indices[-1]):
            if indices[cur_ind]-1==i:
                cur_pred=y_ensemble[cur_ind]
                preds_proba.append(cur_pred)
                cur_ind+=1
            else:
                if cur_ind==0:
                    preds_proba.append(y_ensemble[cur_ind])
                else:
                    w=1-(i-indices[cur_ind-1]+1)/(indices[cur_ind]-indices[cur_ind-1])
                    pred=w*y_ensemble[cur_ind-1]+(1-w)*y_ensemble[cur_ind]
                    preds_proba.append(pred)

        preds_proba=np.array(preds_proba)
        for hInd,delta in enumerate(deltas):
            preds=[]
            for i in range(len(preds_proba)):
                i1=max(i-delta,0)
                proba=np.mean(preds_proba[i1:i+delta+1],axis=0)
                preds.append(np.argmax(proba))
            for i,ind in enumerate(indices):
                if expressions[i]>=0:
                    total_preds[hInd].append(preds[ind-1])

    total_true=np.array(total_true)
    print(weight)
    for hInd,delta in enumerate(deltas):
        preds=np.array(total_preds[hInd])
        print(delta,'Acc:',(preds==total_true).mean(), 'F1:',f1_score(y_true=total_true,y_pred=preds, average="macro"))
    print()

0.0
0 Acc: 0.410110076568805 F1: 0.29090239611199764
5 Acc: 0.4806474840659889 F1: 0.33263223831368627
15 Acc: 0.5046732636561961 F1: 0.34164236744431886
25 Acc: 0.513214178774614 F1: 0.3457739134483474
50 Acc: 0.5193703392126389 F1: 0.3521084905477545
100 Acc: 0.5205038997333638 F1: 0.35453723394331516
200 Acc: 0.5206037100936791 F1: 0.3551542666808812
500 Acc: 0.5236229734932201 F1: 0.3595042852144297

0.1
0 Acc: 0.43592887798896385 F1: 0.3141889443420316
5 Acc: 0.5027733021544779 F1: 0.3563857812809881
15 Acc: 0.522806667332069 F1: 0.36445956300269716
25 Acc: 0.531885845465045 F1: 0.3692206390706086
50 Acc: 0.5371579712831335 F1: 0.374503014447069
100 Acc: 0.5369440919396005 F1: 0.3757623082995216
200 Acc: 0.5277651034463091 F1: 0.36969726350868176
500 Acc: 0.5231132277244663 F1: 0.3658137189900791

0.2
0 Acc: 0.46409678753226014 F1: 0.3439200559542131
5 Acc: 0.5192241883278913 F1: 0.3826124760034294
15 Acc: 0.5344630915546177 F1: 0.3892371889323604
25 Acc: 0.5388476180970442 F1: 0.

In [73]:
#Ensemble
#deltas=[0,5,15,25,50,100,200,500]
thresholds=[0.5, 0.6, 0.7, 0.8, 0.9, 1.0]#np.linspace(0,1,11)
delta=50
for weight in np.linspace(0,1,11):
    total_true=[]
    total_preds=[[] for _ in range(len(thresholds))]
    for videoname,(y_pred_expr,indices,expressions,scores) in test_videos.items():
        for i,ind in enumerate(indices):
            if expressions[i]>=0:
                total_true.append(expressions[i])
        
        y_pred_expr_audio,_,_,_=test_audios[videoname]
        y_ensemble=weight*y_pred_expr + (1-weight)*y_pred_expr_audio
        pretrained_scores_probabs=scores2probabs(scores)
        cur_ind=0
        preds_proba,pretrained_preds_proba=[],[]
        for i in range(indices[-1]):
            if indices[cur_ind]-1==i:
                preds_proba.append(y_ensemble[cur_ind])
                pretrained_preds_proba.append(pretrained_scores_probabs[cur_ind])
                cur_ind+=1
            else:
                if cur_ind==0:
                    preds_proba.append(y_ensemble[cur_ind])
                    pretrained_preds_proba.append(pretrained_scores_probabs[cur_ind])
                else:
                    w=1-(i-indices[cur_ind-1]+1)/(indices[cur_ind]-indices[cur_ind-1])
                    pred=w*y_ensemble[cur_ind-1]+(1-w)*y_ensemble[cur_ind]
                    preds_proba.append(pred)

                    s=w*pretrained_scores_probabs[cur_ind-1]+(1-w)*pretrained_scores_probabs[cur_ind]
                    pretrained_preds_proba.append(s)

        preds_proba=np.array(preds_proba)
        pretrained_preds_proba=np.array(pretrained_preds_proba)

        for hInd,t in enumerate(thresholds):
            preds=[]
            for i in range(len(preds_proba)):
                i1=max(i-delta,0)
                proba=np.mean(preds_proba[i1:i+delta+1],axis=0)
                pred=np.argmax(proba)

                pretrained_proba=np.mean(pretrained_preds_proba[i1:i+delta+1],axis=0)
                pretrained_pred=np.argmax(pretrained_proba[:7])
                if pretrained_proba[pretrained_pred]>t:
                    pred=pretrained_pred
                preds.append(pred)

            for i,ind in enumerate(indices):
                if expressions[i]>=0:
                    total_preds[hInd].append(preds[ind-1])

    total_true=np.array(total_true)
    print(weight)
    for hInd,t in enumerate(thresholds):
        preds=np.array(total_preds[hInd])
        print(t,'Acc:',(preds==total_true).mean(), 'F1:',f1_score(y_true=total_true,y_pred=preds, average="macro"))
    print()        
        

0.0
0.5 Acc: 0.5345593372592075 F1: 0.41248830720304686
0.6 Acc: 0.5548101464360572 F1: 0.42396717744619905
0.7 Acc: 0.5547459826329973 F1: 0.4199674277881535
0.8 Acc: 0.5428115152638558 F1: 0.40376213627384594
0.9 Acc: 0.5273694266607731 F1: 0.37575897320527984
1.0 Acc: 0.51937746852409 F1: 0.35211269341760687

0.1
0.5 Acc: 0.5420771961843925 F1: 0.4175389398022777
0.6 Acc: 0.5646806781401053 F1: 0.4322326958234649
0.7 Acc: 0.5677248941297249 F1: 0.43109571235292327
0.8 Acc: 0.5571129140347625 F1: 0.4161933155374388
0.9 Acc: 0.5431715454921364 F1: 0.3906061112686796
1.0 Acc: 0.5371579712831335 F1: 0.3745089484818034

0.2
0.5 Acc: 0.5391007086535582 F1: 0.4157700687458627
0.6 Acc: 0.5631763934239231 F1: 0.43462188528178486
0.7 Acc: 0.5671973250823436 F1: 0.4352077835045166
0.8 Acc: 0.5583926254402349 F1: 0.42234143934991075
0.9 Acc: 0.5461765502687751 F1: 0.40051848435931264
1.0 Acc: 0.5388440534413186 F1: 0.38897485868167003

0.30000000000000004
0.5 Acc: 0.5373575920037643 F1: 0.41553

In [24]:
#deltas=[0,1,5,7,15]
deltas=[0,5,15,25,50,100,200,500]
total_true=[]
total_preds=[[] for _ in range(len(deltas))]
for videoname,(y_pred_expr,indices,expressions,scores) in test_videos.items():
    for i,ind in enumerate(indices):
        if expressions[i]>=0:
            total_true.append(expressions[i])
    cur_ind=0
    preds_proba=[]
    for i in range(indices[-1]):
        if indices[cur_ind]-1==i:
            preds_proba.append(y_pred_expr[cur_ind])
            cur_ind+=1
        else:
            if cur_ind==0:
                preds_proba.append(y_pred_expr[cur_ind])
            else:
                w=(i-indices[cur_ind-1]+1)/(indices[cur_ind]-indices[cur_ind-1])
                pred=w*y_pred_expr[cur_ind-1]+(1-w)*y_pred_expr[cur_ind]
                preds_proba.append(pred)
    
    preds_proba=np.array(preds_proba)
    for hInd,delta in enumerate(deltas):
        preds=[]
        for i in range(len(preds_proba)):
            i1=max(i-delta,0)
            proba=np.mean(preds_proba[i1:i+delta+1],axis=0)
            best_ind=np.argmax(proba)
            best_ind_no_other=np.argmax(proba[:7])
            if best_ind==7 and proba[best_ind]-proba[best_ind_no_other]<0.05:
                best_ind=best_ind_no_other
            preds.append(best_ind)
        for i,ind in enumerate(indices):
            if expressions[i]>=0:
                total_preds[hInd].append(preds[ind-1])

total_true=np.array(total_true)
for hInd,delta in enumerate(deltas):
    preds=np.array(total_preds[hInd])
    print(delta,'Acc:',(preds==total_true).mean(), 'F1:',f1_score(y_true=total_true,y_pred=preds, average="macro"))

0 Acc: 0.49302753340082417 F1: 0.3830806902655762
5 Acc: 0.5112928293385425 F1: 0.3984449112888923
15 Acc: 0.5252484565040708 F1: 0.4102184763684504
25 Acc: 0.5313440177947614 F1: 0.4164778611208603
50 Acc: 0.5389545577688107 F1: 0.42223307250557535
100 Acc: 0.5359317297135443 F1: 0.41796605670219833
200 Acc: 0.5328589964781202 F1: 0.4164616736756614
500 Acc: 0.5271769352515934 F1: 0.4067049624885466


### Save test results 

In [31]:
with open(os.path.join(DATA_DIR,'test_set_examples/CVPR_6th_ABAW_Expr_test_set_example.txt'),'r') as f:
    test_set_sample=f.read().splitlines()
print(len(test_set_sample),test_set_sample[:5])

test_set_videos={}
for s in test_set_sample[1:]:
    videoname,img_name=s[:-2].split('/')
    if videoname not in test_set_videos:
        test_set_videos[videoname]=[]
    test_set_videos[videoname].append(img_name)
    
print(len(test_set_videos))

1022656 ['image_location,Neutral,Anger,Disgust,Fear,Happiness,Sadness,Surprise,Other', '14-30-1920x1080/00001.jpg,0', '14-30-1920x1080/00002.jpg,0', '14-30-1920x1080/00003.jpg,0', '14-30-1920x1080/00004.jpg,0']
228


In [37]:
#mlpModel.load_weights('expr_enet0_vgaf_train_val.h5')
mlpModel.load_weights('expr_enet0_vgaf_train_val_new.h5')
mlpModelAudio.load_weights('expr_wav2vec_train_val_new.h5')

In [38]:
test_videos={}
test_videos_num_frames={}
for videoname,img_files in tqdm(test_set_videos.items()):
    X,indices,filenames,scores=[],[],[],[]
    num_present=num_missed=0
    for img_name in img_files:
        k=videoname+'/'+img_name
        if k in filename2featuresAll:
            X.append(filename2featuresAll[k][0])
            indices.append(int(img_name[:-4]))
            filenames.append(k)
            scores.append(filename2featuresAll[k][1][AFFECTNET2MTL])
            num_present+=1
        else:
            num_missed+=1
    test_videos[videoname]=(mlpModel.predict(np.array(X),verbose=0),indices,filenames,np.array(scores))
    test_videos_num_frames[videoname]=(num_present,num_missed)
print(len(test_videos))

100%|██████████| 228/228 [01:24<00:00,  2.69it/s]

228


In [34]:
test_audios={}
test_audios_num_frames={}
for videoname,img_files in tqdm(test_set_videos.items()):
    X,indices,filenames,scores=[],[],[],[]
    num_present=num_missed=0
    fn_short=videoname
    if videoname.endswith('_left'):
        fn_short=videoname[:-5]
    elif videoname.endswith('_right'):
        fn_short=videoname[:-6]
    for img_name in img_files:
        k=fn_short+'/'+img_name
        if k in filename2audio_features:
            X.append(filename2audio_features[k])
            indices.append(int(img_name[:-4]))
            filenames.append(k)
            num_present+=1
        else:
            num_missed+=1
    test_audios[videoname]=(mlpModelAudio.predict(np.array(X),verbose=0),indices,filenames,np.array(scores))
    test_audios_num_frames[videoname]=(num_present,num_missed)
print(len(test_audios))

100%|██████████| 228/228 [01:17<00:00,  2.93it/s]

228


In [80]:
if False:
    with open('expr_test_audios.pickle', 'wb') as handle:
        pickle.dump(test_audios, handle, protocol=pickle.HIGHEST_PROTOCOL)
else:
    with open('../abaw6/expr_test_audios.pickle', 'rb') as handle:
        test_audios=pickle.load(handle)
print(len(test_audios))

228


In [35]:
delta=50 # 0 2 4 7
OUTPUT_DIR=DATA_DIR+'/test_results/EXPR/abaw8'
if not os.path.exists(OUTPUT_DIR):
    os.mkdir(OUTPUT_DIR)
print(OUTPUT_DIR)

/home/HDD6TB/datasets/emotions/ABAW/ABAW_8/8th_ABAW_Annotations/test_results/EXPR/abaw8


In [102]:
#for t in np.linspace(0.5,1,6):
for t in [0.7, 0.8, 0.9]:
#for t in [0.9]:
    num_changed_frames,num_frames=0,0
    with open(os.path.join(OUTPUT_DIR,'predictions_'+str(t)+'_train_val.txt'), 'w') as f:
    #with open(os.path.join(OUTPUT_DIR,'predictions_'+str(t)+'.txt'), 'w') as f:
        f.write(test_set_sample[0]+'\n')
        for videoname,(y_pred_expr,indices,filenames,scores) in test_videos.items():
            pretrained_scores_probabs=scores2probabs(scores)
            cur_ind=0
            preds_proba,pretrained_preds_proba=[],[]
            for i in range(indices[-1]):
                if indices[cur_ind]-1==i:
                    preds_proba.append(y_pred_expr[cur_ind])
                    pretrained_preds_proba.append(pretrained_scores_probabs[cur_ind])
                    cur_ind+=1
                else:
                    if cur_ind==0:
                        preds_proba.append(y_pred_expr[cur_ind])
                        pretrained_preds_proba.append(pretrained_scores_probabs[cur_ind])
                    else:
                        w=1-(i-indices[cur_ind-1]+1)/(indices[cur_ind]-indices[cur_ind-1])
                        pred=w*y_pred_expr[cur_ind-1]+(1-w)*y_pred_expr[cur_ind]
                        preds_proba.append(pred)

                        s=w*pretrained_scores_probabs[cur_ind-1]+(1-w)*pretrained_scores_probabs[cur_ind]
                        pretrained_preds_proba.append(s)

            pred=y_pred_expr[cur_ind-1]
            pretrained_pred=pretrained_scores_probabs[cur_ind-1]
            for _ in range(indices[-1],len(test_set_videos[videoname])):
                preds_proba.append(pred)
                pretrained_preds_proba.append(pretrained_pred)

            preds_proba=np.array(preds_proba)
            pretrained_preds_proba=np.array(pretrained_preds_proba)
            for i,img_name in enumerate(test_set_videos[videoname]):
                i1=max(i-delta,0)
                proba=np.mean(preds_proba[i1:i+delta+1],axis=0)
                pred=np.argmax(proba)

                pretrained_proba=np.mean(pretrained_preds_proba[i1:i+delta+1],axis=0)
                pretrained_pred=np.argmax(pretrained_proba[:7])
                if pretrained_proba[pretrained_pred]>t:
                    if pred!=pretrained_pred:
                        num_changed_frames+=1
                    pred=pretrained_pred
                num_frames+=1
                f.write(videoname+'/'+img_name+','+str(pred)+'\n')
    print(num_changed_frames,num_frames)

66661 1022655
27906 1022655
11813 1022655


In [39]:
#weight=0.7
for weight in [0.5, 0.6, 0.7, 0.8, 0.9]:#[0.6, 0.9]:
    for t in [0.5, 0.6, 0.7, 0.8, 0.9]:
        num_changed_frames,num_frames=0,0
        with open(os.path.join(OUTPUT_DIR,'predictions_'+str(weight)+'_'+str(t)+'_train_val_audio.txt'), 'w') as f:
        #with open(os.path.join(OUTPUT_DIR,'predictions_'+str(t)+'.txt'), 'w') as f:
            f.write(test_set_sample[0]+'\n')
            for videoname,(y_pred_expr,indices,filenames,scores) in test_videos.items():
                y_pred_expr_audio,indices_audio,filenames_audio,_=test_audios[videoname]

                pretrained_scores_probabs=scores2probabs(scores)
                cur_ind=0
                preds_proba,pretrained_preds_proba=[],[]
                for i in range(indices[-1]):
                    if indices[cur_ind]-1==i:
                        preds_proba.append(y_pred_expr[cur_ind])
                        pretrained_preds_proba.append(pretrained_scores_probabs[cur_ind])
                        cur_ind+=1
                    else:
                        if cur_ind==0:
                            preds_proba.append(y_pred_expr[cur_ind])
                            pretrained_preds_proba.append(pretrained_scores_probabs[cur_ind])
                        else:
                            w=1-(i-indices[cur_ind-1]+1)/(indices[cur_ind]-indices[cur_ind-1])
                            pred=w*y_pred_expr[cur_ind-1]+(1-w)*y_pred_expr[cur_ind]
                            preds_proba.append(pred)

                            s=w*pretrained_scores_probabs[cur_ind-1]+(1-w)*pretrained_scores_probabs[cur_ind]
                            pretrained_preds_proba.append(s)

                pred=y_pred_expr[cur_ind-1]
                pretrained_pred=pretrained_scores_probabs[cur_ind-1]
                for _ in range(indices[-1],len(test_set_videos[videoname])):
                    preds_proba.append(pred)
                    pretrained_preds_proba.append(pretrained_pred)

                preds_proba=np.array(preds_proba)
                preds_proba=weight*preds_proba+(1-weight)*y_pred_expr_audio
                pretrained_preds_proba=np.array(pretrained_preds_proba)
                for i,img_name in enumerate(test_set_videos[videoname]):
                    i1=max(i-delta,0)
                    proba=np.mean(preds_proba[i1:i+delta+1],axis=0)
                    pred=np.argmax(proba)

                    pretrained_proba=np.mean(pretrained_preds_proba[i1:i+delta+1],axis=0)
                    pretrained_pred=np.argmax(pretrained_proba[:7])
                    if pretrained_proba[pretrained_pred]>t:
                        if pred!=pretrained_pred:
                            num_changed_frames+=1
                        pred=pretrained_pred
                    num_frames+=1
                    f.write(videoname+'/'+img_name+','+str(pred)+'\n')
        print(weight,t,num_changed_frames,num_frames)

0.5 0.5 235868 1022655
0.5 0.6 131933 1022655
0.5 0.7 67209 1022655
0.5 0.8 29048 1022655
0.5 0.9 12420 1022655
0.6 0.5 235307 1022655
0.6 0.6 131752 1022655
0.6 0.7 67043 1022655
0.6 0.8 28616 1022655
0.6 0.9 12163 1022655
0.7 0.5 233706 1022655
0.7 0.6 130129 1022655
0.7 0.7 65598 1022655
0.7 0.8 27326 1022655
0.7 0.9 11475 1022655
0.8 0.5 234807 1022655
0.8 0.6 130724 1022655
0.8 0.7 65907 1022655
0.8 0.8 27464 1022655
0.8 0.9 11581 1022655
0.9 0.5 235781 1022655
0.9 0.6 131263 1022655
0.9 0.7 66223 1022655
0.9 0.8 27614 1022655
0.9 0.9 11690 1022655
